In [1]:
# !pip install pymupdf4llm
# !pip install transformers
# !pip install "pinecone[grpc]"
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import pymupdf4llm
from langchain_text_splitters import RecursiveCharacterTextSplitter

md_text = pymupdf4llm.to_markdown("/home/cristianomendieta/estudos/chat-with-pdf/case_files/LB5001.pdf")

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
            # Set a really small chunk size, just to show.
            chunk_size=1000,
            chunk_overlap=100,
            length_function=len,
            is_separator_regex=False,
        )
texts = text_splitter.create_documents([md_text])

In [4]:
texts

[Document(metadata={}, page_content='**AC & DC Motor Installation & Maintenance Instructions**\n**Handling**\nThe weight of the motor and shipping container will vary. Use correct material handling\nequipment to avoid injury.\n\n**Receiving**\nInspect the motor for damage before accepting it. The Motor shaft should rotate freely with\nno rubs. Report any damage immediately to the commercial carrier that delivered your\nmotor.'),
 Document(metadata={}, page_content='**Safety Notice**\nOnly qualified personnel trained in the safe installation and operation of this equipment\nshould install this motor. When improperly installed or used, rotating equipment can\ncause serious or fatal injury. Equipment must be installed in accordance with the National\nElectrical Code (NEC), local codes and NEMA MG2 Safety Standards for Construction and\nGuide for Selection, Installation and Use of Electric Motors and Generators. Observe the\nfollowing guidelines:\n1. When eyebolts are provided, they must b

In [1]:
from pinecone import Pinecone, ServerlessSpec
from pinecone import ServerlessSpec

pc = Pinecone(api_key="pcsk_2SrBmu_MDxZLJmhrvwasnH9P8KEDAHmKfA5hhGWKebMRaNdRm8yuuf6wcwcsszpBNuvaBL")

dense_index_name = "dense-chat-with-pdf"
sparse_index_name = "sparse-chat-with-pdf"

if not pc.has_index(dense_index_name):
    pc.create_index(
        name=dense_index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ),
        deletion_protection="disabled",
    )

if not pc.has_index(sparse_index_name):
    pc.create_index(
        name=sparse_index_name,
        vector_type="sparse",
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [6]:
from transformers import BertTokenizerFast  # !pip install transformers

# load bert tokenizer from huggingface
tokenizer = BertTokenizerFast.from_pretrained(
   'bert-base-multilingual-uncased'
)

/home/cristianomendieta/estudos/chat-with-pdf/my_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
# tokenize the context passage
inputs = tokenizer(
    texts[0].page_content, padding=True, truncation=True,
    max_length=512
)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [8]:
# extract the input ids
input_ids = inputs['input_ids']
input_ids

[101,
 115,
 115,
 12181,
 111,
 14410,
 13469,
 33040,
 111,
 33379,
 58179,
 115,
 115,
 115,
 115,
 47240,
 115,
 115,
 10103,
 23602,
 10108,
 10103,
 13469,
 10110,
 47639,
 77258,
 11229,
 37355,
 119,
 11868,
 46085,
 13220,
 47240,
 22490,
 10114,
 32406,
 26751,
 119,
 115,
 115,
 30512,
 115,
 115,
 15450,
 84789,
 10103,
 13469,
 10139,
 24046,
 11364,
 39898,
 10285,
 10197,
 119,
 10103,
 13469,
 65315,
 14693,
 46566,
 10218,
 12487,
 10563,
 10171,
 10181,
 13462,
 20540,
 119,
 13100,
 11318,
 24046,
 23062,
 10114,
 10103,
 16229,
 34441,
 10203,
 35768,
 12787,
 13469,
 119,
 102]

In [9]:
from collections import Counter
from langchain_openai import OpenAIEmbeddings

def build_dict(input_batch):
    # store a batch of sparse embeddings
    sparse_emb = []
    # iterate through input batch
    for token_ids in input_batch:
        # convert the input_ids list to a dictionary of key to frequency values
        d = dict(Counter(token_ids))
        # remove special tokens and create indices/values lists
        indices = []
        values = []
        for idx in d:
            if idx not in [101, 102, 103, 0]:  # filter out special tokens
                indices.append(idx)
                values.append(float(d[idx]))
        sparse_emb.append({'indices': indices, 'values': values})
    # return sparse_emb list
    return sparse_emb


def generate_sparse_vectors(context_batch):
    # create batch of input_ids
    inputs = tokenizer(
            context_batch, padding=True,
            truncation=True,
            max_length=512
    )['input_ids']
    # create sparse dictionaries
    sparse_embeds = build_dict(inputs)
    return sparse_embeds


def generate_dense_vectors(context_batch):
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",
    )
    
    # Generate embeddings for the entire batch
    dense_vectors = embeddings.embed_documents(context_batch)
    
    return dense_vectors


In [10]:
text_chunks = [text.page_content for text in texts]

In [11]:
len(text_chunks)

9

In [15]:
# from tqdm.auto import tqdm

# batch_size = 32
# vectors = []  # Initialize vectors list

# for i in tqdm(range(0, len(text_chunks), batch_size)):
#     # find end of batch
#     i_end = min(i+batch_size, len(text_chunks))
#     # extract batch
#     context_batch = text_chunks[i:i_end]
#     # create unique IDs
#     ids = [str(x) for x in range(i, i_end)]
#     # add context passages as metadata
#     meta = [{'chunk_text': context} for context in context_batch]
#     # create sparse vectors
#     sparse_embeds = generate_sparse_vectors(context_batch)

#     # loop through the data and create dictionaries for uploading documents to pinecone index
#     for _id, sparse, metadata in zip(ids, sparse_embeds, meta):
#         vectors.append({
#             'id': _id,
#             'sparse_values': sparse,  # sparse já tem o formato correto: {'indices': [...], 'values': [...]}
#             'metadata': metadata
#         })

# print(f"Total vectors created: {len(vectors)}")
# print(f"First vector format: {vectors[0] if vectors else 'No vectors created'}")

# #     # upload the documents to the new hybrid index
# #     index.upsert(vectors=vectors)

# # # show index description after uploading the documents
# # index.describe_index_stats()

In [12]:
# Processamento de vetores separados para cada índice
from tqdm.auto import tqdm

batch_size = 32
dense_vectors = []  # Vetores apenas densos
sparse_vectors = []  # Vetores apenas esparsos

for i in tqdm(range(0, len(text_chunks), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(text_chunks))
    # extract batch
    context_batch = text_chunks[i:i_end]
    # create unique IDs
    ids = [str(x) for x in range(i, i_end)]
    # add context passages as metadata
    meta = [{'chunk_text': context} for context in context_batch]
    
    # create sparse vectors
    sparse_embeds = generate_sparse_vectors(context_batch)
    # create dense vectors
    dense_embeds = generate_dense_vectors(context_batch)

    # Create separate vectors for each index type
    for _id, sparse, dense, metadata in zip(ids, sparse_embeds, dense_embeds, meta):
        # Dense vector for dense index
        dense_vectors.append({
            'id': _id,
            'values': dense,  # only dense vector
            'metadata': metadata
        })
        
        # Sparse vector for sparse index
        sparse_vectors.append({
            'id': _id,
            'sparse_values': sparse,  # only sparse vector
            'metadata': metadata
        })

print(f"Total dense vectors created: {len(dense_vectors)}")
print(f"Total sparse vectors created: {len(sparse_vectors)}")
print(f"Dense vector dimension: {len(dense_vectors[0]['values']) if dense_vectors else 'N/A'}")
print(f"Sparse vector indices count: {len(sparse_vectors[0]['sparse_values']['indices']) if sparse_vectors else 'N/A'}")

100%|██████████| 1/1 [00:01<00:00,  1.65s/it]

Total dense vectors created: 9
Total sparse vectors created: 9
Dense vector dimension: 1536
Sparse vector indices count: 52


In [13]:
len(dense_vectors[0]['values'])

1536

In [3]:
# Upload dos vetores para os índices apropriados
dense_index = pc.Index(dense_index_name)
sparse_index = pc.Index(sparse_index_name)

/home/cristianomendieta/estudos/chat-with-pdf/my_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print("Upload completed!")
print("\nDense index stats:")
print(dense_index.describe_index_stats())
print("\nSparse index stats:")
print(sparse_index.describe_index_stats())

Upload completed!

Dense index stats:
{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 61}},
 'total_vector_count': 61,
 'vector_type': 'dense'}

Sparse index stats:
{'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {'': {'vector_count': 61}},
 'total_vector_count': 61,
 'vector_type': 'sparse'}


In [6]:
dense_index.delete(delete_all=True)

{}

In [7]:
sparse_index.delete(delete_all=True)

{}

In [ ]:
# Upload dos vetores para os índices apropriados
dense_index = pc.Index(dense_index_name)
sparse_index = pc.Index(sparse_index_name)

# Upload em batches para evitar problemas de timeout
upload_batch_size = 100

print("Uploading dense vectors...")
for i in tqdm(range(0, len(dense_vectors), upload_batch_size)):
    i_end = min(i + upload_batch_size, len(dense_vectors))
    batch_vectors = dense_vectors[i:i_end]
    dense_index.upsert(vectors=batch_vectors)

print("Uploading sparse vectors...")
for i in tqdm(range(0, len(sparse_vectors), upload_batch_size)):
    i_end = min(i + upload_batch_size, len(sparse_vectors))
    batch_vectors = sparse_vectors[i:i_end]
    sparse_index.upsert(vectors=batch_vectors)

In [2]:
print("Upload completed!")
print("\nDense index stats:")
print(dense_index.describe_index_stats())
print("\nSparse index stats:")
print(sparse_index.describe_index_stats())

Upload completed!

Dense index stats:


NameError: name 'dense_index' is not defined

In [17]:
def sparse_query(question, top_k):
    # convert the question into a sparse vector
    sparse_vec = generate_sparse_vectors([question])[0]
    # query pinecone with the query parameters
    result = sparse_index.query(
        sparse_vector=sparse_vec,
        top_k=top_k,
        include_metadata=True
    )
    # return search results as json
    return result

In [18]:
sparse_query_result = sparse_query(
    question="Polyrex EM",
    top_k=5
)

In [21]:
def dense_query(question, top_k):
    # convert the question into a dense vector
    dense_vec = generate_dense_vectors([question])[0]
    # query pinecone with the query parameters
    result = dense_index.query(
        vector=dense_vec,
        top_k=top_k,
        include_metadata=True
    )
    # return search results as json
    return result

In [22]:
dense_query_result = dense_query(
    question="whats the name of the company that produces Polyrex EM?",
    top_k=5
)

In [38]:
# dense_query_result

In [41]:
# sparse_query_result

In [33]:
h1 = sparse_query_result
h2 = dense_query_result

deduped_hits = {hit['id']: hit for hit in h1['matches'] + h2['matches']}.values()

In [42]:
def merge_chunks(h1, h2):
    """Get the unique hits from two search results and return them as single array of {'id', 'chunk_text'} dicts, printing each dict on a new line."""
    # Deduplicate by id
    deduped_hits = {hit['id']: hit for hit in h1['matches'] + h2['matches']}.values()

    # Sort by _score descending
    sorted_hits = sorted(deduped_hits, key=lambda x: x['score'], reverse=True)
    # Transform to format for reranking
    result = [{'id': hit['id'], 'chunk_text': hit['metadata']['chunk_text']} for hit in sorted_hits]
    return result

merged_results = merge_chunks(sparse_query_result, dense_query_result)

print('[\n   ' + ',\n   '.join(str(obj) for obj in merged_results) + '\n]')

[
   {'id': '3', 'chunk_text': '**Motor Enclosure**\nODP, Open drip proof motors are intended for use in clean, dry locations with adequate\nsupply of cooling air. These motors should not be used in the presence of flammable or\ncombustible materials. Open motors can emit flame and/or molten metal in the event of\ninsulation failure.\nTEFC, totally enclosed motors are intended for use where moisture, dirt and/or corrosive\nmaterials are present in indoor and outdoor locations. Explosion proof motors, as indicated\nby the Underwriters Laboratories, Inc. label are intended for use in hazardous areas as\nspecified by the NEC.'},
   {'id': '6', 'chunk_text': '|also recommended at these inte|ervals: Relubrication Intervals|Col3|Col4|Col5|\n|---|---|---|---|---|\n|NEMA (IEC) Frame Size|Rated Speed (RPM)|Rated Speed (RPM)|Rated Speed (RPM)|Rated Speed (RPM)|\n|NEMA (IEC) Frame Size|3600|1800|1200|900|\n|Up to 210 incl. (132)|5500Hrs.|12000Hrs.|18000Hrs.|22000Hrs.|\n|Over 210 to 280 incl. (180

In [ ]:
# Exemplo de como usar as estratégias de busca implementadas na aplicação

# Simulando o uso das estratégias:
print("=== Estratégias de Busca Implementadas ===")
print()

# 1. Busca Dense (usando OpenAI embeddings)
print("1. BUSCA DENSE:")
print("   - Usa embeddings do OpenAI (text-embedding-3-small)")
print("   - Melhor para busca semântica")
print("   - Dimensão: 1536")
print("   - Índice: dense-chat-with-pdf")
print()

# 2. Busca Sparse (usando BERT tokenizer)
print("2. BUSCA SPARSE:")
print("   - Usa tokenização BERT (bert-base-multilingual-uncased)")
print("   - Melhor para correspondência exata de palavras")
print("   - Baseada em frequência de tokens")
print("   - Índice: sparse-chat-with-pdf")
print()

# 3. Busca Híbrida (combinando ambas)
print("3. BUSCA HÍBRIDA:")
print("   - Combina dense + sparse")
print("   - Pesos: 70% dense + 30% sparse (configurável)")
print("   - Melhor dos dois mundos")
print("   - Deduplicação automática")
print()

print("=== Como usar na aplicação ===")
print()
print("# Exemplo de request para a API:")
example_request = {
    "question": "O que é Polyrex EM?",
    "search_strategy": "hybrid",  # dense, sparse, ou hybrid
    "max_documents": 5
}
print(f"POST /questions/")
print(f"Body: {example_request}")
print()

print("# Exemplo de response:")
example_response = {
    "answer": "Resposta baseada nos documentos encontrados...",
    "references": [
        "documento1.pdf (Score: 0.95, Tipo: hybrid)",
        "documento2.pdf (Score: 0.87, Tipo: hybrid)"
    ],
    "search_strategy": "hybrid",
    "documents_found": 5,
    "context_used": 3
}
print(f"Response: {example_response}")

In [12]:
md_text = pymupdf4llm.to_markdown(doc="/home/cristianomendieta/estudos/chat-with-pdf/case_files/WEG-motores-eletricos-guia-de-especificacao-50032749-brochure-portuguese-web.pdf",
                                      write_images=True,
                                      image_path="/home/cristianomendieta/estudos/chat-with-pdf/case_files/images-2")

In [1]:
# !pip install -U sentence-transformers

In [3]:
from sentence_transformers import CrossEncoder

# 1. Load a pre-trained CrossEncoder model
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")

# 2. Predict scores for a pair of sentences
scores = model.predict([
    ("How many people live in Berlin?", "Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers."),
    ("How many people live in Berlin?", "Berlin is well known for its museums."),
])
# => array([ 8.607138 , -4.3200774], dtype=float32)

# 3. Rank a list of passages for a query
query = "How many people live in Berlin?"
passages = [
    "Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.",
    "Berlin is well known for its museums.",
    "In 2014, the city state Berlin had 37,368 live births (+6.6%), a record number since 1991.",
    "The urban area of Berlin comprised about 4.1 million people in 2014, making it the seventh most populous urban area in the European Union.",
    "The city of Paris had a population of 2,165,423 people within its administrative city limits as of January 1, 2019",
    "An estimated 300,000-420,000 Muslims reside in Berlin, making up about 8-11 percent of the population.",
    "Berlin is subdivided into 12 boroughs or districts (Bezirke).",
    "In 2015, the total labour force in Berlin was 1.85 million.",
    "In 2013 around 600,000 Berliners were registered in one of the more than 2,300 sport and fitness clubs.",
    "Berlin has a yearly total of about 135 million day visitors, which puts it in third place among the most-visited city destinations in the European Union.",
]
ranks = model.rank(query, passages)

# Print the scores
print("Query:", query)
for rank in ranks:
    print(f"{rank['score']:.2f}\t{passages[rank['corpus_id']]}")
"""
Query: How many people live in Berlin?
8.92    The urban area of Berlin comprised about 4.1 million people in 2014, making it the seventh most populous urban area in the European Union.
8.61    Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.
8.24    An estimated 300,000-420,000 Muslims reside in Berlin, making up about 8-11 percent of the population.
7.60    In 2014, the city state Berlin had 37,368 live births (+6.6%), a record number since 1991.
6.35    In 2013 around 600,000 Berliners were registered in one of the more than 2,300 sport and fitness clubs.
5.42    Berlin has a yearly total of about 135 million day visitors, which puts it in third place among the most-visited city destinations in the European Union.
3.45    In 2015, the total labour force in Berlin was 1.85 million.
0.33    Berlin is subdivided into 12 boroughs or districts (Bezirke).
-4.24   The city of Paris had a population of 2,165,423 people within its administrative city limits as of January 1, 2019
-4.32   Berlin is well known for its museums.
"""

ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L6-v2')
scores = model.predict([
    ("How many people live in Berlin?", "Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers."),
    ("How many people live in Berlin?", "Berlin is well known for its museums."),
])
print(scores)
# [ 8.607138 -4.320078]


ModuleNotFoundError: No module named 'sentence_transformers'